In [1]:
import pyspark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1628691762731_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
enem = (
    spark
    .read
    .format('csv')
    .option('header', True)
    .option('inferSchema', True)
    .option('delimiter', ';')
    .load('s3://mindsetcloud/datalake/')
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
enem.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- NU_INSCRICAO: long (nullable = true)
 |-- NU_ANO: integer (nullable = true)
 |-- CO_MUNICIPIO_RESIDENCIA: integer (nullable = true)
 |-- NO_MUNICIPIO_RESIDENCIA: string (nullable = true)
 |-- CO_UF_RESIDENCIA: integer (nullable = true)
 |-- SG_UF_RESIDENCIA: string (nullable = true)
 |-- NU_IDADE: integer (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_ESTADO_CIVIL: integer (nullable = true)
 |-- TP_COR_RACA: integer (nullable = true)
 |-- TP_NACIONALIDADE: integer (nullable = true)
 |-- CO_MUNICIPIO_NASCIMENTO: integer (nullable = true)
 |-- NO_MUNICIPIO_NASCIMENTO: string (nullable = true)
 |-- CO_UF_NASCIMENTO: integer (nullable = true)
 |-- SG_UF_NASCIMENTO: string (nullable = true)
 |-- TP_ST_CONCLUSAO: integer (nullable = true)
 |-- TP_ANO_CONCLUIU: integer (nullable = true)
 |-- TP_ESCOLA: integer (nullable = true)
 |-- TP_ENSINO: integer (nullable = true)
 |-- IN_TREINEIRO: integer (nullable = true)
 |-- CO_ESCOLA: integer (nullable = true)
 |-- CO_MU

In [49]:
df = (
    enem
    .select('NU_IDADE', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'SG_UF_RESIDENCIA',
            'NO_MUNICIPIO_RESIDENCIA','NO_MUNICIPIO_NASCIMENTO','TP_NACIONALIDADE','NO_MUNICIPIO_PROVA',
           'IN_DEFICIENCIA_FISICA','TP_PRESENCA_CH','TP_PRESENCA_CN','IN_GESTANTE','IN_CEGUEIRA',
            'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT')
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
from pyspark.sql.functions import mean, max, min, col, count, sum

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
#Sobre o ENEM 2019, qual é a média da nota em matemática apenas para as alunas do sexo Feminino?

(df.groupby('TP_SEXO').agg(mean('NU_NOTA_MT').alias('NOTA_MAT')).show())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+
|TP_SEXO|         NOTA_MAT|
+-------+-----------------+
|      M|547.9751098893438|
|      F| 506.167126424273|
+-------+-----------------+

In [8]:
#Qual é a média da nota em Ciências Humanas para os alunos do sexo masculino que residem no estado de São Paulo?
df.filter((df.SG_UF_RESIDENCIA == 'SP') & ( df.TP_SEXO == 'M')).agg(mean('NU_NOTA_CH').alias('NOTA_CH')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+
|          NOTA_CH|
+-----------------+
|535.2805391773538|
+-----------------+

In [86]:
#Qual é a cidade (município de residência) que possui a MAIOR nota em Ciências Naturais?
#df.groupby('NO_MUNICIPIO_RESIDENCIA').max('NU_NOTA_CN').first().asDict()['max(NU_NOTA_CN)']
#df.agg({'NU_NOTA_CN': 'max'}).show()

#df.withColumn('NU_NOTA_CN', 'NO_MUNICIPIO_RESIDENCIA').max('NU_NOTA_CN').show()(df.where((df.NU_NOTA_CN == max(df.NU_NOTA_CN)).select('NU_NOTA_CN', 'NO_MUNICIPIO_RESIDENCIA').groupby('NU_NOTA_CN', 'NO_MUNICIPIO_RESIDENCIA').agg(max(df.NU_NOTA_CN).alias('NOTA_CN'))))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
col should be Column
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py", line 2456, in withColumn
    assert isinstance(col, Column), "col should be Column"
AssertionError: col should be Column



In [10]:
#Qual é a média de idade dos alunos que NASCERAM na cidade de Natal?

df.filter((df.NO_MUNICIPIO_NASCIMENTO == 'Natal')).agg(mean('NU_IDADE').alias('MEDIA_IDADE')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+
|       MEDIA_IDADE|
+------------------+
|23.498853211009173|
+------------------+

In [24]:
#Qual é o código da escola que obteve a maior nota MÉDIA em matemática?

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
#Quantas pessoas que moram no estado do Rio de Janeiro do sexo masculino obtiveram nota em matemática maior do que 600?
df.filter((df.TP_SEXO == 'M') & (df.NU_NOTA_MT > 600 ) & (df.SG_UF_RESIDENCIA == 'RJ')).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

39551

In [57]:
#Quantas pessoas nascidas em Recife fizeram a prova do ENEM nessa mesma cidade?
df.filter((df.NO_MUNICIPIO_NASCIMENTO == 'Recife') & (df.NO_MUNICIPIO_PROVA == 'Recife')).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

48109

In [48]:
#Quantas mulheres gestantes que moram no estado de SP fizeram a prova de Ciências Humanas do ENEM em 2019?
df.filter((df.IN_GESTANTE == 1 ) & (df.SG_UF_RESIDENCIA == 'SP') & (df.TP_PRESENCA_CH == 1)).groupby('IN_GESTANTE','SG_UF_RESIDENCIA').agg(sum('IN_GESTANTE').alias('TOTAL')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------------+-----+
|IN_GESTANTE|SG_UF_RESIDENCIA|TOTAL|
+-----------+----------------+-----+
|          1|              SP|   34|
+-----------+----------------+-----+

In [39]:
#Qual é a nota média em Ciências Humanas dos alunos que moram no estado de Santa Catarina e possuem deficiência física?
df.filter((df.IN_DEFICIENCIA_FISICA == 1 ) & (df.SG_UF_RESIDENCIA == 'SC')).groupby('IN_DEFICIENCIA_FISICA','SG_UF_RESIDENCIA').agg(mean('NU_NOTA_CH').alias('MEDIA_NOTA_CH')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+----------------+-----------------+
|IN_DEFICIENCIA_FISICA|SG_UF_RESIDENCIA|    MEDIA_NOTA_CH|
+---------------------+----------------+-----------------+
|                    1|              SC|503.0277777777777|
+---------------------+----------------+-----------------+

In [52]:
#Qual é a nota média em matemática dos alunos cegos do sexo feminino residentes em Belo Horizonte?
df.filter((df.TP_SEXO == 'F') & (df.IN_CEGUEIRA == 1 ) & (df.NO_MUNICIPIO_RESIDENCIA == 'Belo Horizonte')).groupby('TP_SEXO','IN_CEGUEIRA').agg(mean('NU_NOTA_MT').alias('MEDIA_MATEMATICA')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------+----------------+
|TP_SEXO|IN_CEGUEIRA|MEDIA_MATEMATICA|
+-------+-----------+----------------+
|      F|          1|           445.5|
+-------+-----------+----------------+